# Домашнее задание Анализ результатов



1. Отфильтруйте датасет по своей гипотезе. Используйте метрику, которая описана в блоке “Ожидаем”
2. Постройте периодический p-value по выбранной метрике (сплит уже произведен)
    * Разбейте общее количество наблюдений на 7 участков (используйте столбец даты)
    * Для каждого участка используйте встроенную функцию ТТЕСТ() в excel, чтобы сравнить тестовую и котрольную группы
    * 7 полученных значений p-value визуализируйте на графике
3. На основе графика сделайте вывод о значимости результатов.


**фичa для анализа:**

Если юзер Android нажимает “отменить”, предлагать промокод в случае НЕ отмены

Ожидаем: ниже отмены на Android устройствах

In [1]:
#подключаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.stats as stats
from scipy.stats import ttest_ind

In [ ]:
#загружаем файл
df = pd.read_csv('post_abtest_data.csv', sep=',')

In [ ]:
df.head(20)

In [ ]:
df.info()

проверим значения в столбцах user_cancelled и city_center_order 

In [ ]:
df['user_cancelled'].unique()

In [ ]:
df['city_center_order'].unique()

т.е. в столбцах user_cancelled и city_center_order значение NaN соответствует данным, по которым не было совершено заказа.

In [ ]:
#фильтр для пользователей андроид, которые делали заказ
df_android = df[(df['os'] == 'Android') & (df['order_made'] == 1)]
df_android

In [ ]:
#группируем по ID, группе теста и дате с агрегацией на количество отмен
df_android = df_android.groupby(['user_id','groups','date']).agg({'user_cancelled':'sum'}).reset_index()
df_android

In [ ]:
# какие даты есть в df_android
df_android.date.unique() 

In [ ]:
#смотрим длину групп 1 и 2
len(df_android[df_android['groups']=='Test']), len(df_android[df_android['groups']=='Control'])

In [ ]:
# делим датасет на 7 частей. Для выборки взяла по 2500 (т.к. 18522/7=2646) для равенства контрольно-тестовых групп.
day_1_control = df_android[(df_android.date == '2020-05-25') & (df_android.groups == 'Control')]['user_cancelled'].dropna().sample(n=2500)
day_1_test = df_android[(df_android.date == '2020-05-25') & (df_android.groups == 'Test')]['user_cancelled'].dropna().sample(n=2500)

In [ ]:
day_2_control = df_android[(df_android.date == '2020-05-26') & (df_android.groups == 'Control')]['user_cancelled'].dropna().sample(n=2500)
day_2_test = df_android[(df_android.date == '2020-05-26') & (df_android.groups == 'Test')]['user_cancelled'].dropna().sample(n=2500)

In [ ]:
day_3_control = df_android[(df_android.date == '2020-05-27') & (df_android.groups == 'Control')]['user_cancelled'].dropna().sample(n=2500)
day_3_test = df_android[(df_android.date == '2020-05-27') & (df_android.groups == 'Test')]['user_cancelled'].dropna().sample(n=2500)

In [ ]:
day_4_control = df_android[(df_android.date == '2020-05-28') & (df_android.groups == 'Control')]['user_cancelled'].dropna().sample(n=2500)
day_4_test = df_android[(df_android.date == '2020-05-28') & (df_android.groups == 'Test')]['user_cancelled'].dropna().sample(n=2500)

In [ ]:
day_5_control = df_android[(df_android.date == '2020-05-29') & (df_android.groups == 'Control')]['user_cancelled'].dropna().sample(n=2500)
day_5_test = df_android[(df_android.date == '2020-05-29') & (df_android.groups == 'Test')]['user_cancelled'].dropna().sample(n=2500)

In [ ]:
day_6_control = df_android[(df_android.date == '2020-05-30') & (df_android.groups == 'Control')]['user_cancelled'].dropna().sample(n=2500)
day_6_test = df_android[(df_android.date == '2020-05-30') & (df_android.groups == 'Test')]['user_cancelled'].dropna().sample(n=2500)

In [ ]:
day_7_control = df_android[(df_android.date == '2020-05-31') & (df_android.groups == 'Control')]['user_cancelled'].dropna().sample(n=2500)
day_7_test = df_android[(df_android.date == '2020-05-31') & (df_android.groups == 'Test')]['user_cancelled'].dropna().sample(n=2500)

In [ ]:
#проверяем длину
len(day_1_control), len(day_1_test)

In [ ]:
# заводим списки для удобства перебирания в цикле
test = [day_1_test,day_2_test,day_3_test,day_4_test,day_5_test,day_6_test,day_7_test] 

In [ ]:
control = [day_1_control,day_2_control,day_3_control,day_4_control,day_5_control,day_6_control,day_7_control] 

In [ ]:
# рассчитываем p-value
Pvalue = []
for i in range(7):
    T, p = stats.ttest_ind(control[i], test[i])
    Pvalue.append(p)

In [ ]:
# p-value списком по дням
Pvalue 

In [ ]:
# строим график по p-value по дням, где красная линия это alpha, а синяя - периодический p-value
alpha=0.05

fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(111)
ax.plot(sorted(df_android['date'].unique()), Pvalue)
ax.plot(sorted(df_android['date'].unique()), [alpha]*7, color='red')

**ВЫВОД:** p-value стабильно ниже alpha, следовательно, предложение промокода пользователям Android при нажатии кнопки отмены заказа приводит к сокращению отмен. Фича работает.